In [1]:
!pip install -q transformers huggingface_hub
!pip install -q --upgrade accelerate
!pip install -q -U bitsandbytes
!pip install -q torch

In [2]:
import pandas as pd

# Define the bucket and file names
bucket_name = 'agentsum'  # Replace with your bucket name
mimic_iv_bhc = f's3://{bucket_name}/sample_data_100.csv'

# Load the files
mimic_iv_bhc_100 = pd.read_csv(mimic_iv_bhc)

# Display the data
mimic_iv_bhc_100.head(1)

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/fsspec/registry.py:294: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


,note_id,input,target,input_tokens,target_tokens
0,16002318-DS-17,<SEX> F <SERVICE> SURGERY <ALLERGIES> Iodine /...,This is a ___ yo F admitted to the hospital af...,1195,75


In [3]:
from huggingface_hub import login
from huggingface_hub import whoami
import getpass

# Prompt the user for the Hugging Face token at runtime
hf_token = getpass.getpass("Enter your Hugging Face token: ")

# Log in using the provided token
login(token=hf_token)


print(whoami(token=hf_token))

Enter your Hugging Face token:  ········


{'type': 'user', 'id': '6644f24fe7ae8316ebf3fee4', 'name': 'LizaPiya', 'fullname': 'Fahmida Liza Piya', 'email': 'lizapiya@udel.edu', 'emailVerified': True, 'canPay': False, 'periodEnd': None, 'isPro': False, 'avatarUrl': '/avatars/d43d60b3eba464c3f9b44c34e43b64d6.svg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'Clinical Note LLama', 'role': 'write', 'createdAt': '2024-06-03T19:29:07.142Z'}}}


In [4]:
!pip install jax jaxlib flax

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Try a different clinical T5 model
model_name = "google/flan-t5-base"  # General T5 that works well
# OR try: "facebook/bart-large-cnn" for summarization

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def generate_summary(clinical_note):
    prompt = f"Summarize this clinical note in about 200 words: {clinical_note}"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=300,
            min_length=100,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            early_stopping=True
        )
    
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary


def process_mimic_data(df):
    results = []
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Generating summaries"):
        try:
            summary = generate_summary(row['input'])
            summary_tokens = len(tokenizer.encode(summary))
            results.append({
                'note_id': row['note_id'],
                'original_input': row['input'],
                'generated_summary': summary,
                'target_summary': row['target'],
                'summary_token_count': summary_tokens,
                'input_tokens': row['input_tokens'],
                'target_tokens': row['target_tokens']
            })
        except Exception as e:
            results.append({
                'note_id': row['note_id'],
                'original_input': row['input'],
                'generated_summary': "ERROR: Could not generate summary",
                'target_summary': row['target'],
                'summary_token_count': 0,
                'input_tokens': row['input_tokens'],
                'target_tokens': row['target_tokens']
            })
    
    return pd.DataFrame(results)

# Run the pipeline
print("Loading and processing entire MIMIC dataset...")
print(f"Total samples to process: {len(mimic_iv_bhc_100)}")
full_results = process_mimic_data(mimic_iv_bhc_100)

# CHANGE 7: Update filename
full_results.to_csv('clinical_t5_summaries_full_dataset.csv', index=False)
print(f"\nAll results saved to 'clinical_t5_summaries_full_dataset.csv'")

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Loading and processing entire MIMIC dataset...
Total samples to process: 100


Generating summaries: 100%|██████████| 100/100 [09:28<00:00,  5.68s/it]


All results saved to 'clinical_t5_summaries_full_dataset.csv'


In [2]:
%run evaluate_Flan_T5.py

🚀 Starting Flan-T5 MIMIC Evaluation
📂 Loaded 100 samples from Flan_t5_summaries_full_dataset.csv
📋 Data columns: ['note_id', 'original_input', 'generated_summary', 'target_summary', 'summary_token_count', 'input_tokens', 'target_tokens']
📏 Data shape: (100, 7)
📊 Evaluating 100 Flan-T5-generated summaries...
📝 Summary column: generated_summary
🎯 Reference column: target_summary

🔢 Computing BLEU and ROUGE-L scores...


Processing Rows: 100%|██████████| 100/100 [00:01<00:00, 70.79it/s]



🧠 Computing BERTScore...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You sho


📊 FLAN-T5 EVALUATION RESULTS
Metric       Mean ± Std           Min      Max      Median  
----------------------------------------------------------------------
BLEU1        4.510 ± 6.912    0.00     36.62    0.73    
BLEU2        2.202 ± 4.354    0.00     28.83    0.27    
ROUGE_L      7.818 ± 6.860    0.00     34.71    6.62    
BERT_P       79.254 ± 6.512    57.82    89.57    81.20   
BERT_R       78.199 ± 2.945    72.31    89.16    78.17   
BERT_F1      78.636 ± 4.381    64.26    86.82    79.52   

📈 BASELINE METRICS SUMMARY (for table)
BLEU-1: 4.51 ± 6.91
BLEU-2: 2.20 ± 4.35
ROUGE-L: 7.82 ± 6.86
BERTScore-F1: 78.64 ± 4.38

📏 TOKEN LENGTH ANALYSIS
Generated Summary Tokens:
  Mean ± Std: 199.9 ± 82.6
  Target: 150 tokens
  Range: 5 - 300
  Within 140-160: 9/100 (9.0%)

Target Summary Tokens:
  Mean ± Std: 581.1 ± 450.6
  Range: 18 - 2659

💾 Results saved to: Flan_t5_summaries_full_dataset_evaluation_results.csv

📋 SAMPLE RESULTS
Note ID: 16002318-DS-17
Generated Summary Token Count:

### llm as a judge

In [3]:
import sys
sys.path.append("..")
# Run the entire script
%run llm_as_a_judge_flant5.py

Running Flan-T5 evaluation...
🔄 Loading Llama 3 8B model as judge...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

📄 Loaded 100 Flan-T5 generated summaries
📋 Columns: ['note_id', 'original_input', 'generated_summary', 'target_summary', 'summary_token_count', 'input_tokens', 'target_tokens']
🔍 Evaluating Flan-T5 summaries for hallucinations...


Evaluating: 100%|██████████| 100/100 [08:52<00:00,  5.33s/it]


📊 FLAN-T5 HALLUCINATION EVALUATION RESULTS:
Metric                    Mean ± Std      Min    Max    Perfect Scores
----------------------------------------------------------------------
Hallucination (1-5)       2.37 ± 0.80   1.0    5.0    2/100
Factual Consistency (1-5) 3.79 ± 0.62   1.0    5.0    1/100
Completeness (1-5)        3.18 ± 0.80   1.0    5.0    9/100
Coherence (1-5)           4.60 ± 1.03   1.0    5.0    85/100

📋 BASELINE QUALITY INSIGHTS:
• High hallucination (≥4): 6/100 (6.0%)
• Low factual consistency (≤2): 4/100 (4.0%)
• Good completeness (≥4): 20/100 (20.0%)
• Good coherence (≥4): 86/100 (86.0%)

📊 FOR BASELINE TABLE:
Hallucination: 2.37 ± 0.80
Factual Consistency: 3.79 ± 0.62
Completeness: 3.18 ± 0.80
Coherence: 4.60 ± 1.03

💾 Results saved to: flan_t5_judge_results.csv
Done!
